In [1]:
import collections
import fasttext
import json
import nltk
import pickle
import random
import scipy.sparse
import sys
import time

import numpy as np
import pandas as pd
import zstandard as zstd

from collections import Counter
from langdetect import detect
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from scipy.sparse import dok_matrix

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import LongType, StructField, StructType
from pyspark.ml.clustering import LDA, LDAModel, LocalLDAModel
from pyspark.ml.linalg import Vectors, SparseVector

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/olam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Remove video that have 0 tokens in the new vocab

In [4]:
S = scipy.sparse.load_npz('/dlabdata1/youtube_large/olam/matrices/S_final2.npz')

In [9]:
row_with_tokens = []
for i in range(S.shape[0]):
    if S[i, :].count_nonzero() != 0:
        row_with_tokens.append(i)

In [10]:
len(row_with_tokens)

68566847

In [11]:
S

<68638982x663127 sparse matrix of type '<class 'numpy.uint8'>'
	with 1251231562 stored elements in Compressed Sparse Row format>

In [12]:
scipy.sparse.save_npz('/dlabdata1/youtube_large/olam/matrices/S_final3.npz', S[row_with_tokens,:])

## Get list of index of relevant videos, from each categorie, each year and each channel


In [9]:
class Zreader:

    def __init__(self, file, chunk_size=16384):
        '''Init method'''
        self.fh = open(file,'rb')
        self.chunk_size = chunk_size
        self.dctx = zstd.ZstdDecompressor()
        self.reader = self.dctx.stream_reader(self.fh)
        self.buffer = ''


    def readlines(self):
        '''Generator method that creates an iterator for each line of JSON'''
        while True:
            chunk = self.reader.read(self.chunk_size).decode(errors="ignore")
            if not chunk:
                break
            lines = (self.buffer + chunk).split("\n")

            for line in lines[:-1]:
                yield line

            self.buffer = lines[-1]

In [10]:
# Load set of videos to consider
with open('/dlabdata1/youtube_large/olam/filtered10000/idx_vid_to_consider.pickle', 'rb') as f:
    idx_vid_to_consider = pickle.load(f)
f.close()

In [11]:
n_rows = len(idx_vid_to_consider)
n_columns = 5

columns_names = ['idx', 'channel_id', 'view_counts', 'uploaded_year', 'category']

In [12]:
array_relevant_infos = []

In [13]:
reader = Zreader("/dlabdata1/youtube_large/yt_metadata_all.jsonl.zst", chunk_size=2**28)

In [14]:
start = time.time()
idx = 0
idx_new = 0
for line in reader.readlines():
    ###start_iter = time.time()
    idx += 1
    
    if idx % 1000000 == 0:
        print('Progress: ' + str(int(idx/1000000)) + '/85')
        
    if idx in idx_vid_to_consider:
        
        # line is a str dict, video is the dict corresponding to the str dict
        video = json.loads(line)
        array_vid_relevant_infos = [idx_new + 1]
        
        array_vid_relevant_infos.append(video['channel_id'])
        array_vid_relevant_infos.append(video['view_count'])
        array_vid_relevant_infos.append(video['upload_date'][:4])
        array_vid_relevant_infos.append(video['categories'])
        
        array_relevant_infos.append(array_vid_relevant_infos)
        
        idx_new += 1
        
print(str(time.time() - start))

Progress: 1/85
Progress: 2/85
Progress: 3/85
Progress: 4/85
Progress: 5/85
Progress: 6/85
Progress: 7/85
Progress: 8/85
Progress: 9/85
Progress: 10/85
Progress: 11/85
Progress: 12/85
Progress: 13/85
Progress: 14/85
Progress: 15/85
Progress: 16/85
Progress: 17/85
Progress: 18/85
Progress: 19/85
Progress: 20/85
Progress: 21/85
Progress: 22/85
Progress: 23/85
Progress: 24/85
Progress: 25/85
Progress: 26/85
Progress: 27/85
Progress: 28/85
Progress: 29/85
Progress: 30/85
Progress: 31/85
Progress: 32/85
Progress: 33/85
Progress: 34/85
Progress: 35/85
Progress: 36/85
Progress: 37/85
Progress: 38/85
Progress: 39/85
Progress: 40/85
Progress: 41/85
Progress: 42/85
Progress: 43/85
Progress: 44/85
Progress: 45/85
Progress: 46/85
Progress: 47/85
Progress: 48/85
Progress: 49/85
Progress: 50/85
Progress: 51/85
Progress: 52/85
Progress: 53/85
Progress: 54/85
Progress: 55/85
Progress: 56/85
Progress: 57/85
Progress: 58/85
Progress: 59/85
Progress: 60/85
Progress: 61/85
Progress: 62/85
Progress: 63/85
P

In [15]:
df = pd.DataFrame(array_relevant_infos, columns=columns_names)

In [16]:
df.shape

(21714294, 5)

In [17]:
df.head(5)

,idx,channel_id,view_counts,uploaded_year,category
0,1,UCzzzZ3-icktxbC3j7hkWqRw,1888967,2016,Howto & Style
1,2,UCzzzZ3-icktxbC3j7hkWqRw,1297474,2016,Howto & Style
2,3,UCzzzZ3-icktxbC3j7hkWqRw,582615,2016,Howto & Style
3,4,UCzzzZ3-icktxbC3j7hkWqRw,14507,2016,Howto & Style
4,5,UCzzzZ3-icktxbC3j7hkWqRw,171671,2016,Howto & Style


In [18]:
len(idx_vid_to_consider)

21714294

In [19]:
df_top5 = df.sort_values(['view_counts'], ascending=False).groupby(['category', 'uploaded_year', 'channel_id']).head(5)
df_top10 = df.sort_values(['view_counts'], ascending=False).groupby(['category', 'uploaded_year', 'channel_id']).head(10)                                                                                                                   
df_top20 = df.sort_values(['view_counts'], ascending=False).groupby(['category', 'uploaded_year', 'channel_id']).head(20)

In [20]:
sorted_idx_relevant_vid_top5 = sorted(df_top5.index.values)
sorted_idx_relevant_vid_top10 = sorted(df_top10.index.values)
sorted_idx_relevant_vid_top20 = sorted(df_top20.index.values)

In [21]:
with open('/dlabdata1/youtube_large/olam/filtered10000/sorted_idx_relevant_vid_top5.pickle', 'wb') as f:
    pickle.dump(sorted_idx_relevant_vid_top5, f)
f.close()
with open('/dlabdata1/youtube_large/olam/filtered10000/sorted_idx_relevant_vid_top10.pickle', 'wb') as f:
    pickle.dump(sorted_idx_relevant_vid_top10, f)
f.close()
with open('/dlabdata1/youtube_large/olam/filtered10000/sorted_idx_relevant_vid_top20.pickle', 'wb') as f:
    pickle.dump(sorted_idx_relevant_vid_top20, f)
f.close()

In [22]:
len(sorted_idx_relevant_vid_top5)

3432979

In [23]:
len(sorted_idx_relevant_vid_top10)

5453143

In [24]:
len(sorted_idx_relevant_vid_top20)

8116217

In [1]:
7728523 / 68000000

0.11365475

In [2]:
7728523 / 45421300

0.17015195513998937

# Save file to work on hadoop cluster

In [2]:
conf = SparkConf().setMaster("local[4]").setAll([('spark.executor.memory', '4g'),('spark.driver.memory','16g'),('spark.driver.maxResultSize', '0')])

# create the session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# create the context
sc = spark.sparkContext

In [26]:
def get_dict_for_row(row, S):
    '''Construct SparseVector bag-of-word for each row (videos)'''
    tmp_dict = {}
    for key, value in row:
        tmp_dict[key[1]] = value

    return SparseVector(S.shape[1], tmp_dict)

def remove_zero_rows(M):
    '''Function that removes all rows from sparse matrix M that contains only zero.'''
    num_nonzeros = np.diff(M.indptr)
    return M[num_nonzeros != 0]

In [30]:
# Load data
print('Loading data...')
S = scipy.sparse.load_npz('/dlabdata1/youtube_large/olam/data/view10000_sub10000/csr_matrices/S_final_tok100vid.npz')

# Load set of videos to consider
with open('/dlabdata1/youtube_large/olam/data/view10000_sub10000/sorted_idx_relevant_vid_top20.pickle', 'rb') as f:
    sorted_idx_relevant_vid_top = pickle.load(f)
f.close()

# Select videos
S = S[sorted_idx_relevant_vid_top,:]
S = remove_zero_rows(S)


all_data = []

print('Process video for topic modelling...')
for i in range(S.shape[0]):

    if i % 1000000 == 0:
        print(str(i) + ' videos processed...')

    all_data.append([i, get_dict_for_row(S.getrow(i).todok().items(), S)])
    
    
# Construct dataframe for LDA
all_df = spark.createDataFrame(all_data, ["id", "features"])

Loading data...
Process video for topic modelling...
0 videos processed...
1000000 videos processed...
2000000 videos processed...


In [31]:
all_df.count()

2042687

In [32]:
#save the dataframe
all_df.write\
        .option('compression', 'gzip')\
        .json('/dlabdata1/youtube_large/olam/data/view10000_sub10000/LDA_models/top20/sparkdf.json')

In [16]:
all_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- features: vector (nullable = true)



In [4]:
all_df_test = spark.read.json('/dlabdata1/youtube_large/olam/data/view10000_sub10000/LDA_models/top10/sparkdf.json')

In [7]:
all_df_test.printSchema()

root
 |-- features: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- id: long (nullable = true)



In [55]:
df_collect = all_df_test.collect()

In [8]:
data = []
for row in all_df_test.collect():
    features = row['features']
    data.append([row['id'], SparseVector(features['size'], features['indices'], features['values'])])

KeyboardInterrupt: 

In [68]:
df_test = spark.createDataFrame(data, ['id', 'features'])

In [69]:
df_test.printSchema()

root
 |-- id: long (nullable = true)
 |-- features: vector (nullable = true)



In [70]:
lda = LDA(k=10, seed=1)
model = lda.fit(df_test)

In [72]:
describe_topics = model.describeTopics()

In [ ]:
describe_topics.show()

In [49]:
row_test = df_collect[0]['features']

In [52]:
row_test

Row(indices=[12768, 12772, 42897, 47518, 55859, 72328, 77508, 78050, 87151, 125552, 136529, 150635, 156780, 161250, 166006], size=166209, type=0, values=[1.0, 1.0, 3.0, 1.0, 1.0, 3.0, 1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 1.0, 2.0, 2.0])

In [53]:
SparseVector(row_test['size'], row_test['indices'], row_test['values'])

SparseVector(166209, {12768: 1.0, 12772: 1.0, 42897: 3.0, 47518: 1.0, 55859: 1.0, 72328: 3.0, 77508: 1.0, 78050: 1.0, 87151: 1.0, 125552: 1.0, 136529: 3.0, 150635: 1.0, 156780: 1.0, 161250: 2.0, 166006: 2.0})

In [28]:
all_df_test.printSchema()

root
 |-- features: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- id: long (nullable = true)



In [29]:
all_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- features: vector (nullable = true)



In [33]:
all_df.select('features').show(5, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                  |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(416311,[5881,16455,18197,77164,152293,162655,185041,203528,225256,295597,356334,375392,377662,386188],[2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,3.0])                                                                         |
|(416311,[3414,5881,8565,18197,34688,62895,110100,14

In [3]:
S_view1000_sub10000 = scipy.sparse.load_npz('/dlabdata1/youtube_large/olam/filtered/csr_matrices/S_final_tok100vid.npz')

In [6]:
S_view1000_sub10000

<45421300x166209 sparse matrix of type '<class 'numpy.uint8'>'
	with 473957128 stored elements in Compressed Sparse Row format>

In [4]:
S_view1000_sub100000 = scipy.sparse.load_npz('/dlabdata1/youtube_large/olam/filtered/csr_matrices_100000sub/S_final_tok100vid.npz')
S_view1000_sub100000

<24839929x74362 sparse matrix of type '<class 'numpy.uint8'>'
	with 146450642 stored elements in Compressed Sparse Row format>

In [8]:
S_view10000_sub10000 = scipy.sparse.load_npz('/dlabdata1/youtube_large/olam/filtered10000/csr_matrices/S_final_tok100vid.npz')
S_view10000_sub10000

<21714294x65907 sparse matrix of type '<class 'numpy.uint8'>'
	with 109510371 stored elements in Compressed Sparse Row format>

In [7]:
S_view10000_sub100000 = scipy.sparse.load_npz('/dlabdata1/youtube_large/olam/filtered10000/csr_matrices_100000sub/S_final_tok100vid.npz')
S_view10000_sub100000

<15167437x42757 sparse matrix of type '<class 'numpy.uint8'>'
	with 58140783 stored elements in Compressed Sparse Row format>

##  Remove duplicates of sparse matrix

In [19]:
def remove_duplicate_rows(data):
    unique_row_indices, unique_columns = [], []
    
    for row_idx, row in enumerate(data):
        
        indices = row.indices.tolist()
        
        if indices not in unique_columns:
            
            unique_columns.append(indices)
            unique_row_indices.append(row_idx)
            
    return data[unique_row_indices]

In [4]:
nrows = 10
ncolumns = 5

S_1 = dok_matrix((nrows, ncolumns))

In [9]:
S_1.items()

dict_items([((0, 0), 1.0), ((1, 0), 1.0)])

In [7]:
S_1[0,0] = 1

In [17]:
S_1[1,0] = 2

In [22]:
S_1.todense()

matrix([[1., 0., 0., 0., 0.],
        [2., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [23]:
remove_duplicate_rows(S_1.tocsr()).todense()

matrix([[1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

=top - 10:10:33 up 20 days,  1:54, 10 users,  load average: 35.16, 35.14, 35.06
Tasks: 609 total,   7 running, 355 sleeping,   0 stopped,   1 zombie
%Cpu(s): 19.6 us,  1.4 sy,  1.9 ni, 77.1 id,  0.0 wa,  0.0 hi,  0.0 si,  0.0 st
KiB Mem : 26404467+total, 27091272 free, 15692536+used, 80028032 buff/cache
KiB Swap: 18777268 total,  3514504 free, 15262764 used. 10526962+avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND     
23854 aarora    20   0 8372404 5.930g   3792 R  3000  2.4  34864:23 deepwalk    
24981 jouven    20   0  960640 212172  29276 R 100.0  0.1   1118:19 python      
33702 leone     20   0 36.286g 0.029t  38824 R 100.0 11.8 781:18.79 python3     
43290 olam      20   0 4227200 854292  59824 R 100.0  0.3  83:11.38 python      
43426 olam      20   0 9030908 6.112g  52636 R 100.0  2.4  64:33.82 python      
45222 leone     20   0 44.403g 0.037t  36888 R  94.4 15.1  16:33.10 python3     
45671 olam      20   0   54672   4612   3600 R  11

    2 root      20   0       0      0      0 S   0.0  0.0   0:02.04 kthreadd    top - 10:10:45 up 20 days,  1:54, 10 users,  load average: 35.12, 35.13, 35.06

%Cpu(s): 71.5 us,  1.6 sy,  0.0 ni, 26.9 id,  0.0 wa,  0.0 hi,  0.0 si,  0.0 st
KiB Mem : 26404467+total, 26400680 free, 15756272+used, 80081264 buff/cache
KiB Swap: 18777268 total,  3514504 free, 15262764 used. 10463124+avail Mem 


23854 aarora    20   0 8372404 5.930g   3792 R  3000  2.4  34870:27 deepwalk    
43426 olam      20   0 9032916 6.115g  52636 R 100.3  2.4  64:45.85 python      
24981 jouven    20   0  960648 203464  29276 R 100.0  0.1   1118:31 python      
33702 leone     20   0 36.349g 0.029t  38824 R 100.0 11.9 781:30.92 python3     
43290 olam      20   0 4227968 854920  59824 R 100.0  0.3  83:23.49 python      
45222 leone     20   0 44.953g 0.038t  36888 R 100.0 15.3  16:45.23 python3     
 9459 lugeon    20   0   40816   5976   3852 S   3.6  0.0 110:36.94 htop        
45594 olam      20   0  566204  79592  

## Investigate describe topics

In [2]:
conf = SparkConf().setMaster("local").setAll([
     ('spark.executor.memory', '8g'),
     ('spark.driver.memory','8g'),
     ('spark.driver.maxResultSize', '0')
    ])

# create the session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# create the context
sc = spark.sparkContext

In [ ]:
# Load dictionnary of words
with open('/dlabdata1/youtube_large/olam/data/final_res/id2word_top20.pickle', 'rb') as f:
    id2word = pickle.load(f)
f.close()

In [4]:
describe_topics = spark.read.json('/dlabdata1/youtube_large/olam/data/final_res/model/describe_topics/describe_topics_50.json')

# Print topics
for row in describe_topics.sort('topic').rdd.collect():
    print('Topic ' + str(row.topic) + ': ')
    term_weights = row.termWeights
    for i, token_id in enumerate(row.termIndices):
        print('   With weight of ' + str(term_weights[i]) + ' : ' + id2word[token_id] )
        if i >= 10:
            break

Topic 0: 
   With weight of 0.021964742481566206 : motiv
   With weight of 0.017486007606575767 : peopl
   With weight of 0.015295011491467973 : fnaf
   With weight of 0.015191821508867812 : life
   With weight of 0.014421873711546912 : five
   With weight of 0.014276740530228517 : freddi
   With weight of 0.013805438325242074 : chang
   With weight of 0.013599324987454637 : video
   With weight of 0.013561985845023447 : thoma
   With weight of 0.013331570221141127 : mind
   With weight of 0.013135679676690756 : self
Topic 1: 
   With weight of 0.04709050200803078 : movi
   With weight of 0.03308435773099864 : trailer
   With weight of 0.02882147873402468 : episod
   With weight of 0.026702808251759153 : show
   With weight of 0.02574414298582608 : film
   With weight of 0.016893090966226973 : season
   With weight of 0.015025255363292744 : full
   With weight of 0.013987274304730415 : interview
   With weight of 0.0138186928765321 : scene
   With weight of 0.01205987388663775 : talk
 

In [5]:
describe_topics = spark.read.json('/dlabdata1/youtube_large/olam/data/final_res/model/describe_topics/describe_topics_55.json')

# Print topics
for row in describe_topics.sort('topic').rdd.collect():
    print('Topic ' + str(row.topic) + ': ')
    term_weights = row.termWeights
    for i, token_id in enumerate(row.termIndices):
        print('   With weight of ' + str(term_weights[i]) + ' : ' + id2word[token_id] )
        if i >= 10:
            break

Topic 0: 
   With weight of 0.07588479463794723 : night
   With weight of 0.041516839741706606 : motiv
   With weight of 0.028676998923309983 : fnaf
   With weight of 0.028321403622744323 : five
   With weight of 0.02640202421359214 : freddi
   With weight of 0.026071551473312585 : thoma
   With weight of 0.024862497007476194 : peopl
   With weight of 0.024785930747358877 : video
   With weight of 0.02428753629009413 : michael
   With weight of 0.02216561450654523 : bad
   With weight of 0.019895930539900544 : rich
Topic 1: 
   With weight of 0.057285004670153905 : movi
   With weight of 0.03830782117555569 : trailer
   With weight of 0.03331639949901544 : episod
   With weight of 0.03182292079839711 : show
   With weight of 0.03070086427849224 : film
   With weight of 0.01905635556665852 : season
   With weight of 0.018981172550929057 : full
   With weight of 0.015466461217586946 : scene
   With weight of 0.015022745941909983 : interview
   With weight of 0.013886698396234189 : seri
 

   With weight of 0.0341162328402546 : lego
   With weight of 0.029756478421068072 : app
   With weight of 0.02402078143064236 : review
   With weight of 0.02167967647098531 : phone
   With weight of 0.020002770247213378 : lil
   With weight of 0.01785531324901014 : appl
Topic 47: 
   With weight of 0.10906736301208608 : black
   With weight of 0.06284902453306551 : call
   With weight of 0.04700376640092737 : op
   With weight of 0.04234545899529687 : zombi
   With weight of 0.03686022341731453 : duti
   With weight of 0.023670851586148154 : hero
   With weight of 0.022885888159430597 : cod
   With weight of 0.018480802940087523 : warfar
   With weight of 0.018294768456241735 : faze
   With weight of 0.014403417626478498 : gameplay
   With weight of 0.014291717502521397 : modern
Topic 48: 
   With weight of 0.10960377983049147 : song
   With weight of 0.05612212643054788 : movi
   With weight of 0.03151430664519156 : new
   With weight of 0.02157839841432966 : video
   With weight of 

In [6]:
describe_topics = spark.read.json('/dlabdata1/youtube_large/olam/data/final_res/model/describe_topics/describe_topics_85.json')

# Print topics
for row in describe_topics.sort('topic').rdd.collect():
    print('Topic ' + str(row.topic) + ': ')
    term_weights = row.termWeights
    for i, token_id in enumerate(row.termIndices):
        print('   With weight of ' + str(term_weights[i]) + ' : ' + id2word[token_id] )
        if i >= 10:
            break

Topic 0: 
   With weight of 0.12468624991896424 : night
   With weight of 0.04948465394685785 : peopl
   With weight of 0.04628482287305153 : five
   With weight of 0.04424011664368321 : fnaf
   With weight of 0.04041611716694532 : freddi
   With weight of 0.03997822269070335 : thoma
   With weight of 0.023148249281701463 : french
   With weight of 0.021978601363088896 : market
   With weight of 0.01928686003212564 : locat
   With weight of 0.017054553074615206 : golden
   With weight of 0.01681755635554328 : crime
Topic 1: 
   With weight of 0.12864597000344563 : comedi
   With weight of 0.05255161765102243 : funni
   With weight of 0.03178801464937774 : parodi
   With weight of 0.026626896776572293 : video
   With weight of 0.026309163315319902 : tom
   With weight of 0.02029828921168714 : joke
   With weight of 0.020084280008787148 : show
   With weight of 0.019350207984249235 : stand
   With weight of 0.01586874194331979 : golf
   With weight of 0.015151085765049397 : poni
   With 

In [7]:
describe_topics = spark.read.json('/dlabdata1/youtube_large/olam/data/final_res/model/describe_topics/describe_topics_110.json')

# Print topics
for row in describe_topics.sort('topic').rdd.collect():
    print('Topic ' + str(row.topic) + ': ')
    term_weights = row.termWeights
    for i, token_id in enumerate(row.termIndices):
        print('   With weight of ' + str(term_weights[i]) + ' : ' + id2word[token_id] )
        if i >= 10:
            break

Topic 0: 
   With weight of 0.07804531015980438 : peopl
   With weight of 0.0631326872424876 : bad
   With weight of 0.057955593393987974 : five
   With weight of 0.0535023754786128 : michael
   With weight of 0.053040096276207774 : fnaf
   With weight of 0.048455435306445284 : freddi
   With weight of 0.042115719436807 : night
   With weight of 0.0369662803925542 : sister
   With weight of 0.034528055688828024 : market
   With weight of 0.02897096996005018 : french
   With weight of 0.025942469891543776 : locat
Topic 1: 
   With weight of 0.16430215001574947 : comedi
   With weight of 0.0620624689892826 : funni
   With weight of 0.03998128288814284 : parodi
   With weight of 0.02955272171274831 : video
   With weight of 0.025462653363997414 : joke
   With weight of 0.02379907768288117 : stand
   With weight of 0.01946881510626754 : golf
   With weight of 0.01860647862752941 : humor
   With weight of 0.016931843965064964 : show
   With weight of 0.014965532797490152 : sketch
   With we

   With weight of 0.01651880090576658 : math
   With weight of 0.01459913853678658 : problem
   With weight of 0.014593792129939043 : physic
   With weight of 0.014544456377281644 : current
Topic 104: 
   With weight of 0.1379295353366406 : time
   With weight of 0.11117458246848896 : ball
   With weight of 0.09994681581592295 : dragon
   With weight of 0.04139178384247787 : chris
   With weight of 0.037708683499344114 : brown
   With weight of 0.03609868402221575 : cri
   With weight of 0.03283874320991157 : super
   With weight of 0.021250952042697762 : far
   With weight of 0.01833656969347899 : goku
   With weight of 0.010839938347211866 : cash
   With weight of 0.010450628896956428 : xenovers
Topic 105: 
   With weight of 0.2896251672731749 : hindi
   With weight of 0.027042686698393825 : urdu
   With weight of 0.02676606692491743 : bank
   With weight of 0.01967542882399332 : account
   With weight of 0.019652695322408122 : kais
   With weight of 0.019227736378808914 : hai
   Wit

In [25]:
def get_dict_for_row(row, S):
    '''Construct SparseVector bag-of-word for each row (videos)'''
    tmp_dict = {}
    for key, value in row:
        tmp_dict[key[1]] = value

    return SparseVector(S.shape[1], tmp_dict)

In [26]:
S = scipy.sparse.load_npz('/dlabdata1/youtube_large/olam/data/final_res/matrices/S_final.npz')

In [27]:
data = []

print('Process video for topic modelling...')
for i in range(1000):

    if i % 1000000 == 0:
        print(str(i) + ' videos processed...')

    data.append([i, get_dict_for_row(S.getrow(i).todok().items(), S)])


# Construct dataframe for LDA
all_df = spark.createDataFrame(data, ["id", "features"])

Process video for topic modelling...
0 videos processed...


In [43]:
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA

lda = LDA(k=10, seed=1, topicConcentration=0.01, docConcentration=[0.1, 0.5, 0.9, 0.95, 1])

In [44]:
model = lda.fit(all_df)

IllegalArgumentException: requirement failed: LDA docConcentration was of length 5, but k = 10.  docConcentration must be an array of length either 1 (scalar) or k (num topics).

In [40]:
model.getDocConcentration()

KeyError: Param(parent='LDA_d7cfa2e3922a', name='docConcentration', doc='Concentration parameter (commonly named "alpha") for the prior placed on documents\' distributions over topics ("theta").')